In [11]:
from fastapi import FastAPI, Query
from fastapi.middleware.cors import CORSMiddleware
import requests

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
# Replace with your actual API key
API_URL="https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/silchar/today?unitGroup=us&key=4RH3KGN9E38EDN8W7XJT42SQ5&contentType=json"
@app.get("/weather/")
def get_weather(location: str = Query(..., description="City or location name")):
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/today?unitGroup=us&key=4RH3KGN9E38EDN8W7XJT42SQ5&contentType=json"


    response = requests.get(url)

    if response.status_code != 200:
        return {"error": "Failed to fetch weather data"}

    data = response.json()
    return data['days']


In [36]:
import pandas as pd
data=pd.DataFrame(get_weather(location="guwahati"))

In [37]:
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler



# Handle the input data

# Handle categorical variables (conditions)
conditions_dummies = pd.get_dummies(data['conditions'], prefix='cond')
data = pd.concat([data, conditions_dummies], axis=1)

drop_cols = ['name', 'datetime', 'sunrise', 'sunset', 'conditions', 'description', 
             'icon', 'stations', 'preciptype', 'severerisk','source','hours']


data = data.drop(columns=drop_cols, errors='ignore')
# Handle missing values
# data = data.fillna(data.mean())
#Scale the data

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

print(data, data_scaled)

   datetimeEpoch  tempmax  tempmin  temp  feelslikemax  feelslikemin  \
0     1749407400     94.9     80.5  87.5         110.1          86.9   

   feelslike   dew  humidity  precip  ...  pressure  cloudcover  visibility  \
0       97.5  75.8      69.7     0.0  ...    1001.6        39.7         5.0   

   solarradiation  solarenergy  uvindex  sunriseEpoch  sunsetEpoch  moonphase  \
0           331.9         28.8     10.0    1749423611   1749473075       0.43   

   cond_Partially cloudy  
0                   True  

[1 rows x 27 columns] [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]]


In [34]:
data.columns

Index(['datetimeEpoch', 'tempmax', 'tempmin', 'temp', 'feelslikemax',
       'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'pressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy',
       'uvindex', 'sunriseEpoch', 'sunsetEpoch', 'moonphase', 'source',
       'hours', 'cond_Partially cloudy'],
      dtype='object')

In [21]:
# Load the saved model
from xgboost import XGBRegressor
loaded_model = joblib.load('best_xgb_model.pkl')
new_predictions = loaded_model.predict(data)
print(new_predictions)


ValueError: Feature shape mismatch, expected: 36, got 26